In [8]:
from sklearn.svm import SVC
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df_train = pd.read_csv('data\\train.csv')
# df_train.to_parquet('train.parquet')
# df_train = pd.read_parquet('train.parquet')
df_ts = df_train.iloc[:,5:] # timeseries features only
df_train['y_s'] = df_train.y > 1 # simplified y: label 0 for 1 and 1 for 2,3 

# fit model

In [9]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import make_pipeline
from util import FourierTransformer


params = {'C' : np.linspace(0.1,3,50),
          'kernel':['poly','rbf','linear']}


In [11]:
pipe = make_pipeline(
    FourierTransformer(),
    SVC(class_weight='balanced')
)

pipe.steps

[('fouriertransformer', FourierTransformer()),
 ('svc', SVC(class_weight='balanced'))]

In [16]:
params = {
    'svc__C' : np.linspace(0.5,2,4),
    # 'svc__kernel':['poly','rbf','linear'],
    'fouriertransformer__k': [i for i in range(3,16,3)],
    'fouriertransformer__aggregate': ['union', 'intersection', 'median', 'exclusive']
    }

In [18]:
skf = StratifiedKFold(n_splits=4)
cv = GridSearchCV(
    pipe, 
    param_grid=params, 
    scoring='balanced_accuracy', 
    n_jobs=-1, 
    cv=skf.split(df_ts, df_train.y)
)
cv.fit(df_ts.values,df_train.y)

cv.best_params_, cv.best_score_